In [ ]:
# === Environment Setup ===
import logging
import sys
from IPython.display import display, Markdown

# --- Utility Functions ---
def note(msg): display(Markdown(f"<div class='alert alert-block alert-info'>📝 **Note:** {msg}</div>"))
def sec(title): print(f"\n{80*'='}\n| {title.upper()} |\n{80*'='}")
note("Environment initialized.")

# Chapter 8.20: From Research to Production

---
### Introduction: Beyond the Notebook

The skills required to produce a high-quality research paper are not the same as those required to build a robust, reliable software system. This chapter bridges that gap, providing an introduction to the best practices required to turn the models developed in a research context into **production-ready** applications that can be deployed, monitored, and trusted in a live environment.

**Why does this matter for economists?**
- **Industry Careers:** Economists at tech firms, banks, and hedge funds are often tasked with deploying their models to make real-time decisions (e.g., dynamic pricing, fraud detection, algorithmic trading).
- **Robust Research:** These practices improve the reliability and reproducibility of even purely academic research code.
- **Scalability:** Production principles are essential for running large-scale simulations on the cloud or HPC clusters.

This chapter will cover three pillars of production code: **Logging**, **API Design**, and **Continuous Integration/Continuous Deployment (CI/CD)**.

### 1. Logging: Your Model's Diary

`print()` statements are for interactive exploration. For any code that runs non-interactively, **logging** is the professional standard. A good logging setup provides a structured, configurable, and permanent record of your application's behavior.

**Key Advantages of Logging:**
- **Different Severity Levels:** You can log messages at different levels (e.g., `DEBUG`, `INFO`, `WARNING`, `ERROR`, `CRITICAL`), allowing you to filter for important events.
- **Configurability:** You can easily redirect log output to a file, the console, or a remote service without changing your code.
- **Structured Information:** Logs can be configured to automatically include timestamps, the function name, and the line number where the event occurred, making debugging much easier.

In [ ]:
sec("Example of a Professional Logging Setup")

logger = logging.getLogger('MyEconomicModel')
logger.setLevel(logging.DEBUG) # Set the lowest level to capture all messages

# Create a handler to write logs to the console
handler = logging.StreamHandler(sys.stdout)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Add the handler to the logger
if not logger.handlers:
    logger.addHandler(handler)

logger.info("Model initialization started.")
try:
    x = 1 / 0
except ZeroDivisionError:
    logger.error("An attempt to divide by zero occurred.", exc_info=True)
logger.debug("This is a detailed debug message for developers.")
note("Check the console output to see the structured log messages.")

### 2. API Design: Serving Your Model

A research script might be run manually. A production model needs an **Application Programming Interface (API)** so that other services can interact with it programmatically. For economic models, this often means creating a web API where a user can send a request with new data and receive the model's prediction or forecast in response.

**FastAPI** is a modern, high-performance Python framework for building APIs. It is easy to use and provides automatic data validation and documentation.

**(Conceptual Example - Code not runnable in notebook)**
```python
# main.py
from fastapi import FastAPI
from pydantic import BaseModel
import joblib # For loading a pre-trained model

app = FastAPI()
model = joblib.load('my_model.pkl')

class ModelInput(BaseModel):
    feature1: float
    feature2: int

@app.post('/predict')
def predict(data: ModelInput):
    prediction = model.predict([[data.feature1, data.feature2]])
    return {'prediction': prediction[0]}
```
You would run this with a server like `uvicorn main:app --reload`.

### 3. CI/CD: Automating Quality and Deployment

**Continuous Integration (CI)** and **Continuous Deployment (CD)** are practices that automate the process of testing and deploying code. We have already seen the CI part in **Chapter 1.2** with GitHub Actions running our `pytest` suite.

- **Continuous Integration:** Every time code is pushed to the repository, an automated pipeline runs to build the code, run tests, and check for quality issues. This ensures that bugs are caught early.
- **Continuous Deployment:** If the CI pipeline passes, a CD pipeline can automatically deploy the new version of the application to a staging or production environment. This allows for rapid, reliable releases.

A full CI/CD pipeline for an economic model might look like this:
1.  **Commit:** A developer pushes a change to the model's code.
2.  **CI Pipeline (GitHub Actions):**
    a.  Install dependencies from `environment.yml`.
    b.  Run `pytest` to check for correctness.
    c.  Run `pre-commit` hooks to check for style.
    d.  (Optional) Build a Docker container for the application.
3.  **CD Pipeline:**
    a.  If CI passes on the `main` branch, automatically deploy the new version to a cloud server (e.g., AWS, Google Cloud).